In [1]:
import requests
import cython
import pandas as pd

import os
import os.path
import netCDF4 as nc#not directly used but needs to be imported for some nc4 files manipulations, use for nc files
from netCDF4 import Dataset
import numpy as np
import numpy.ma as ma

In [2]:
# register information from csv file
all_urls = pd.read_csv(r'C:\Users\CLMRX\OneDrive - COWI\Documents\GitHub\CRVA_tool\outputs\NEX-GDDP-CMIP6\gddp-cmip6-thredds-fileserver.csv')

In [3]:
### make all elements of the csv into a readable list

# transpose csv
temp_list = all_urls[[' fileUrl']].T
temp_list=temp_list.values.tolist()
temp_list=temp_list[0]
url_list=[s.replace(' ', '') for s in temp_list]

In [28]:
## download only precipitation data
# select only precipitation files, between 2040 and 2080
url_list_precipitation = [url for url in url_list if 'pr_day_' in url and int(url[len(url)-7:len(url)-3])>2039 and int(url[len(url)-7:len(url)-3])<2081]

['https://ds.nccs.nasa.gov/thredds2/fileServer/AMES/NEX/GDDP-CMIP6/ACCESS-CM2/ssp245/r1i1p1f1/pr/pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2040.nc',
 'https://ds.nccs.nasa.gov/thredds2/fileServer/AMES/NEX/GDDP-CMIP6/ACCESS-CM2/ssp245/r1i1p1f1/pr/pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2041.nc',
 'https://ds.nccs.nasa.gov/thredds2/fileServer/AMES/NEX/GDDP-CMIP6/ACCESS-CM2/ssp245/r1i1p1f1/pr/pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2042.nc',
 'https://ds.nccs.nasa.gov/thredds2/fileServer/AMES/NEX/GDDP-CMIP6/ACCESS-CM2/ssp245/r1i1p1f1/pr/pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2043.nc',
 'https://ds.nccs.nasa.gov/thredds2/fileServer/AMES/NEX/GDDP-CMIP6/ACCESS-CM2/ssp245/r1i1p1f1/pr/pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2044.nc',
 'https://ds.nccs.nasa.gov/thredds2/fileServer/AMES/NEX/GDDP-CMIP6/ACCESS-CM2/ssp245/r1i1p1f1/pr/pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_2045.nc',
 'https://ds.nccs.nasa.gov/thredds2/fileServer/AMES/NEX/GDDP-CMIP6/ACCESS-CM2/ssp245/r1i1p1f1/pr/pr_day_ACCESS-CM2_ssp245_r1i1p1f1_gn_20

In [ ]:
# download only precipitation files
for file in url_list_precipitation: # for loop to download the file in each url
    # find the name of the file
    index_before_name=file.rfind('/') # returns the highest index where the last character '/' was found, which is just before the name of the file    
    f_name = file[index_before_name+1:len(file)-3] # return the name of the file as a string
    # check if the file was already downloaded
    test = os.path.join('//COWI.net/projects/A245000/A248363/CRVA/Datasets/NEX-GDDP-CMIP6/',f_name+'.nc')
    if os.path.isfile(test): # if the file was aleready download
        continue # continue the for loop without executing the code after this line. The code follonwing download the file
        # if we entered the if, the file was already dowloaded, no need to downloaded it again
    # the file was not downloaded, the following code will permit to download it in the servor for dataset
    r = requests.get(file) # return the url were data need to be downloaded
    # download data in the servor for datasets
    with open(f'//COWI.net/projects/A245000/A248363/CRVA/Datasets/NEX-GDDP-CMIP6/{f_name}.nc', 'wb') as f:
        f.write(r.content)

In [ ]:
## read data, _FillValue should be NaN
solar_1950=Dataset(r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\NEX-GDDP-CMIP6\hurs_day_ACCESS-CM2_historical_r1i1p1f1_gn_1950.nc')
hurs=solar_1950.variables['hurs']
hurs
hurs._FillValue
solar_dataframe = np.ma.getdata(solar_1950.variables['hurs']).data
solar_dataframe
len(solar_dataframe[solar_dataframe!=1.e+20])
len(solar_dataframe[solar_dataframe==1.e+20])
type(solar_dataframe)
solar_dataframe.size
solar_dataframe.size-len(solar_dataframe[solar_dataframe!=1.e+20])-len(solar_dataframe[solar_dataframe==1.e+20])